In [2]:
!pip install optuna

  Using cached optuna-4.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.17.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.5.0-py3-none-any.whl (400 kB)
Using cached alembic-1.17.0-py3-none-any.whl (247 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.8 MB/s eta 0:00:00a 0:00:01
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached mako-1.3.10-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [optuna]2m4/5 [optuna]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
import time

# --- 1. Data Loading ---
file_path = '/Users/adityasharma/Github Projects/Amazon/input/train.csv'
try:
    df = pd.read_csv(file_path)
    df = df.dropna(subset=['price'])
    df['catalog_content'] = df['catalog_content'].astype(str).fillna('')
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: File not found at {file_path}. Halting execution.")
    raise

# --- 2. Expensive Pre-computation (Done Once) ---
print("Performing one-time text vectorization...")
start_time = time.time()
vectorizer = TfidfVectorizer(stop_words='english', max_features=30000, ngram_range=(1, 2))
X_transformed = vectorizer.fit_transform(df['catalog_content'])
y = df['price']
y_log = np.log1p(y)
end_time = time.time()
print(f"Vectorization complete in {end_time - start_time:.2f} seconds.")

# --- 3. SMAPE Scorer ---
# This function is now called inside our manual loop.
def smape(y_true, y_pred_log):
    y_pred = np.expm1(y_pred_log)
    y_pred[y_pred < 0] = 0
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    score = np.mean(
        np.divide(numerator, denominator, out=np.zeros_like(numerator, dtype=float), where=denominator!=0)
    ) * 100
    return score

# --- 4. Define the CORRECTED Objective Function ---
def objective(trial):
    params = {
        'objective': 'regression_l1', 'metric': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 42, 'n_jobs': -1, 'verbose': -1
    }
    
    model = lgb.LGBMRegressor(**params)
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    # --- Manual Cross-Validation Loop ---
    for train_index, val_index in kf.split(X_transformed):
        # Split data for this fold
        X_train, X_val = X_transformed[train_index], X_transformed[val_index]
        y_log_train, y_log_val = y_log.iloc[train_index], y_log.iloc[val_index]
        y_true_val = y.iloc[val_index] # Get original price for scoring

        # Train the model
        model.fit(X_train, y_log_train)
        
        # Predict and score
        preds_log = model.predict(X_val)
        fold_score = smape(y_true_val, preds_log)
        scores.append(fold_score)
    
    return np.mean(scores)

# --- 5. Run the Optimization Study ---
study = optuna.create_study(direction='minimize')
print("Starting hyperparameter tuning with corrected CV loop...")
study.optimize(objective, n_trials=50, show_progress_bar=True)

# --- 6. Output the Results ---
print("\n--- Tuning Complete ---")
print(f"Best cross-validated SMAPE Score: {study.best_value:.4f}")
print("\nBest Parameters Found:")
for key, value in study.best_params.items():
    print(f"- {key}: {value}")

Data loaded successfully.
Performing one-time text vectorization...


[I 2025-10-12 15:26:21,439] A new study created in memory with name: no-name-c2bde4f9-3df2-4c16-8eac-b727df3188f0


Vectorization complete in 11.38 seconds.
Starting hyperparameter tuning with corrected CV loop...


  0%|          | 0/50 [00:00<?, ?it/s]

/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


...

[I 2025-10-12 18:23:01,001] Trial 47 finished with value: 52.21092088402602 and parameters: {'n_estimators': 897, 'learning_rate': 0.19762232875570604, 'num_leaves': 30, 'max_depth': 10, 'lambda_l1': 8.389707154875977, 'lambda_l2': 0.8757676042749918, 'feature_fraction': 0.6574917544985645, 'bagging_fraction': 0.9155544518379228, 'bagging_freq': 2, 'min_child_samples': 19}. Best is trial 41 with value: 51.85776044202393.


/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[I 2025-10-12 18:26:49,251] Trial 48 finished with value: 51.80433515928136 and parameters: {'n_estimators': 761, 'learning_rate': 0.1884930281597436, 'num_leaves': 41, 'max_depth': 17, 'lambda_l1': 0.040587974507707486, 'lambda_l2': 2.538242194369503e-06, 'feature_fraction': 0.7302844764184871, 'bagging_fraction': 0.8115111618141515, 'bagging_freq': 5, 'min_child_samples': 9}. Best is trial 48 with value: 51.80433515928136.


/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/adityasharma/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[I 2025-10-12 18:29:20,471] Trial 49 finished with value: 52.38887911869534 and parameters: {'n_estimators': 802, 'learning_rate': 0.1869305575488479, 'num_leaves': 43, 'max_depth': 17, 'lambda_l1': 0.0037698516760839734, 'lambda_l2': 4.9996883433047355e-08, 'feature_fraction': 0.7667582949325866, 'bagging_fraction': 0.8054205164576473, 'bagging_freq': 6, 'min_child_samples': 38}. Best is trial 48 with value: 51.80433515928136.

--- Tuning Complete ---
Best cross-validated SMAPE Score: 51.8043

Best Parameters Found:
- n_estimators: 761
- learning_rate: 0.1884930281597436
- num_leaves: 41
- max_depth: 17
- lambda_l1: 0.040587974507707486
- lambda_l2: 2.538242194369503e-06
- feature_fraction: 0.7302844764184871
- bagging_fraction: 0.8115111618141515
- bagging_freq: 5
- min_child_samples: 9
